In [55]:
import requests
import os

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

In [2]:
url = "https://experienceleague.adobe.com/fr/docs/experience-manager-cloud-service/content/assets/file-format-support"
port = 59795
headless = True
chrome_options = Options()

In [23]:
if port:
    chrome_options.add_argument(f'--remote-debugging-port={port}')
if headless:
    chrome_options.add_argument('--headless')

# Créer une instance du navigateur web Chrome
service = Service()
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=chrome_options)

# Charger la page
driver.get(url)

# Attendre que la page se charge complètement (vous pouvez ajuster le temps d'attente en fonction de la vitesse de chargement de la page)
driver.implicitly_wait(30)

# Obtenir le contenu de la page après que JavaScript ait été exécuté
html = driver.page_source

In [31]:
# Fermer le navigateur
driver.quit()

In [30]:
print(html)

<html lang="fr"><head>
    <title>Formats de fichiers et types MIME pris en charge | Adobe Experience Manager</title>
    <link rel="canonical" href="https://experienceleague.adobe.com/fr/docs/experience-manager-cloud-service/content/assets/file-format-support">
    <meta name="description" content="Formats de fichier et types MIME pris en charge par Experience Manager Assets as a Cloud Service.">
    <meta property="og:title" content="Formats de fichiers et types MIME pris en charge | Adobe Experience Manager">
    <meta property="og:description" content="Formats de fichier et types MIME pris en charge par Experience Manager Assets as a Cloud Service.">
    <meta property="og:url" content="https://experienceleague.adobe.com/fr/docs/experience-manager-cloud-service/content/assets/file-format-support">
    <meta property="og:image" content="https://experienceleague.adobe.com/default-meta-image.png?width=1200&amp;format=pjpg&amp;optimize=medium">
    <meta property="og:image:secure_url" 

In [37]:
# Parser le HTML avec BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

In [41]:
print(soup.find_all("div", {"class": "default-content-wrapper"}))

[<div class="default-content-wrapper"><h1 id="supported-file-formats">Formats de fichiers pris en charge Assets </h1></div>, <div class="default-content-wrapper"><p>Adobe Experience Manager as a Cloud Service prend en charge les fonctionnalités de base de gestion de contenu (stockage, gestion des métadonnées en ligne, contrôle de version, chargement et téléchargement, etc.) pour tout fichier binaire, quel que soit son format. Adobe Experience Manager Assets prend en charge un large éventail de formats de fichiers et chaque fonctionnalité de produit prend en charge différents formats.</p><p>En outre, Experience Manager Assets offre une prise en charge étendue pour générer des aperçus et des rendus et extraire des métadonnées et du texte en vue de l’indexation en texte intégral. Cette prise en charge étendue est assurée à l’aide de <a href="/fr/docs/experience-manager-cloud-service/content/assets/manage/asset-microservices-configure-and-use">microservices de ressources</a>.</p><p>Les élé

In [79]:
def extract_format_and_table(soup, format_type):
    div_adobe_formats = soup.find("h2", {"id": format_type})
    table = None

    if div_adobe_formats:
        format_name = div_adobe_formats.text.strip().title()
        apostrophe_index = format_name.find("’")
        if apostrophe_index:
            # Supprimer la lettre avant l'apostrophe et l'apostrophe elle-même
            format_name_cleaned = format_name[:apostrophe_index-1] + format_name[apostrophe_index+1:]
        
    if div_adobe_formats:
        table = div_adobe_formats.find_next("div", {"class": "table"})
        if table:
            headers = [header.text.strip() for header in table.find("div").find_all("div")]
            return format_name_cleaned, headers, table

    return None, None, None

In [82]:
def extract_data_from_table(table, headers):
    data = []

    for row in table.find_all("div", recursive=False)[1:]:
        # Get the cells of the row
        cells = [cell.text.strip() for cell in row.find_all("div")]
        # Create a dictionary with headers as keys and cells as values
        row_data = dict(zip(headers, cells))
        # Add row data to the list of data
        data.append(row_data)

    return data


In [71]:
def create_folder_if_not_exists(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

In [92]:
def create_folders_from_data(data_formats, format_name):
    folder = 'Formats Type'
    for item in data_formats:
        folder_name = f"{folder}/{format_name}/{item['Format de fichier']}"
        create_folder_if_not_exists(folder_name)

Format Name: Formats Image
Headers: ['Format de fichier', 'Génération de miniatures', 'Extraction de métadonnées', 'Largeur/Hauteur', 'Recadrer']


In [96]:
format_name, headers, table = extract_format_and_table(soup, 'audio-formats')
if format_name and headers:
    print("Format Name:", format_name)
    print("Headers:", headers)
    
table_data = extract_data_from_table(table, headers)
create_folders_from_data(table_data, format_name)

Format Name: Formats AudFormats Audio
Headers: ['Extension de fichier vidéo', 'Conteneur', 'Codecs vidéo recommandés', 'Codecs vidéo non pris en charge']


KeyError: 'Format de fichier'